# 23 May 2021 - MKMCXX Demo

- This is a short demo on how to use the Python interface to MKCMXX to create TOF plots and other contour plots.

In [ ]:
import os
import glob
import pickle
import re
import copy
import datetime

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import mkmcxx

import tqdm
import multiprocessing
from ipywidgets import interact

In [ ]:
# Other resources
%load_ext blackcellmagic

# Autoreload functionality for iterating on packages.
# Uncomment if running cells gets too slow.
%load_ext autoreload
%autoreload 2

Other utility functions:

In [ ]:
def load_from_file(fname, module=pickle):
    """
    Quick function to load an object from a file.

    By default, use the pickle module.
    """

    with open(fname, "rb") as f:
        obj = module.load(f)

    return obj


def save_to_file(obj, fname, module=pickle):
    """
    Quick function to save a single object to a file.

    By default, use the pickle module.
    """

    with open(fname, "wb") as f:
        module.dump(obj, f)

    return True

In [ ]:
# Thanks to https://scentellegher.github.io/visualization/2018/05/02/custom-fonts-matplotlib.html
# specify the custom font to use
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "Nimbus Sans,Arial"
plt.rcParams["mathtext.fontset"] = "custom"
plt.rcParams["font.size"] = 22
plt.rcParams["axes.linewidth"] = 3
plt.rcParams["axes.titlepad"] = 15

# Customize ticks
plt.rcParams["xtick.bottom"] = True
plt.rcParams["xtick.top"] = True
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.left"] = True
plt.rcParams["ytick.right"] = True
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 6
plt.rcParams["xtick.major.width"] = 3
plt.rcParams["ytick.major.size"] = 6
plt.rcParams["ytick.major.width"] = 3

## Basic MKMCXX simulation - Reversible surface isomerization

In [ ]:
# First need to define species
species = {
    "A": mkmcxx.Compound(name="A", is_site=False, start_conc=1.0, tdrc=False),
    "B": mkmcxx.Compound(name="B", is_site=False, start_conc=0, tdrc=False),
    "*": mkmcxx.Compound(name="*", is_site=True, start_conc=1.0, tdrc=False),
    "A*": mkmcxx.Compound(name="A*", is_site=True, start_conc=0, tdrc=False),
    "B*": mkmcxx.Compound(name="B*", is_site=True, start_conc=0, tdrc=False),
}

In [ ]:
# Also define reactions
reactions = [
    mkmcxx.HKReaction(
        reactants={species["A"]: 1, species["*"]: 1},
        products={species["A*"]: 1},
        m2=1e-27,
        amu=1.4e1,
        K=1.50,
        sigma=2,
        sticking=0.015,
        jmol=150,
        do_drc=True,
    ),
    mkmcxx.HKReaction(
        reactants={species["B"]: 1, species["*"]: 1},
        products={species["B*"]: 1},
        m2=1e-22,
        amu=1.4e1,
        K=2.87,
        sigma=2,
        sticking=0.015,
        jmol=200,
        do_drc=True,
    ),
    mkmcxx.ARReaction(
        reactants={species["A*"]: 1},
        products={species["B*"]: 1},
        vf=1e13,
        vb=1e13,
        Eaf=100,
        Eab=600,
        do_drc=True,
    ),
]

In [ ]:
# Define settings and sequence runs
# See settings docs at https://wiki.mkmcxx.nl/index.php/Keywords_and_settings
settings = {
    "type": "sequencerun",    
    "abstol": 1e-12,
    "reltol": 1e-12,
    "reagents": ["A", "*", "A*"],
    "keycomponents": ["A", "*", "A*"],
    # "heatmap": 1,
    # "npar": 2,
    # "drc": 1,
    # "makeplots": 0,
}

# Want to do sequence runs at different temperatures
runs = [{"temp": temp, "time": 1e12} for temp in range(300, 1000, 100)]

In [ ]:
# Create simulation object
sim = mkmcxx.MicrokineticSimulation(
    reactions=reactions,
    settings=settings,
    runs=runs,
    restart=False,
    directory="test-sim",
)

In [ ]:
# Actually run simulations
# You can leave out `args` if mkmcxx is on your $PATH
returncode, stdout, error = sim.run(
    args="/home/samueldy/software-images/mkmcxx_2.15.3.sif mkmcxx -i input.mkm".split()
)
# print(stdout.decode())

# You can also just do sim.write_input_files() to simply write out input folder.

In [ ]:
# Read results. Can do this without having called sim.run() first, which
# enables reading of previously completed results.
results = sim.read_results()

In [ ]:
# Plot how rate of A consumption reaction changes as function of temperature
results["range_results"]["rates"].set_index(keys="Temperature")["A + * -> A*"].plot(
    marker=".", markersize=20
)

In [ ]:
# Also look at how interconversion of A* and B* change with temperature
results["range_results"]["rates"].plot(
    x="Temperature", y=["A* -> B*", "A* <- B*"], marker=".", markersize=10
)

## Integrate adsorbate scaling and BEP relationships

- Let's say that A and B are intermediates in a reaction mechanism where the thermodynamic descriptors are O and N binding energy (EO and EN) and the applied potential U.
- Now, the reaction set will change dynamically as a function of EO, EN, and U.
- Define your reaction set with adsorbate scaling and BEP relationships that are functions of EO, EN, and U.
- Then programmatically run a grid of simulations.

In [ ]:
# First need to define species
species = {
    "A": mkmcxx.Compound(name="A", is_site=False, start_conc=1.0, tdrc=False),
    "B": mkmcxx.Compound(name="B", is_site=False, start_conc=0, tdrc=False),
    "*": mkmcxx.Compound(name="*", is_site=True, start_conc=1.0, tdrc=False),
    "A*": mkmcxx.Compound(name="A*", is_site=True, start_conc=0, tdrc=False),
    "B*": mkmcxx.Compound(name="B*", is_site=True, start_conc=0, tdrc=False),
}

### Generate reaction sets for each pair of binding energies

In [ ]:
# Define your reaction set in terms of adsorption energies and an applied potential
# You can see for yourself how the barriers and adsorption energies change
@interact(EO=(-100, 5, 5), EN=(-100, 5, 5), U=(-0.5, 0.5, 0.1))
def generate_reactions(EO: float, EN: float, U: float):
    """
    EO and EN are adsorption energies of O and N, respectively, in kJ/mol.
    U is the applied potential in V vs. SHE.
    """
    reactions = [
        mkmcxx.HKReaction(
            reactants={species["A"]: 1, species["*"]: 1},
            products={species["A*"]: 1},
            m2=1e-27,
            amu=1.4e1,
            K=1.50,
            sigma=2,
            sticking=0.015,
            jmol=100 + 1.2 * EO - 1.0 * EN + 0.5 * U,
            do_drc=True,
        ),
        mkmcxx.HKReaction(
            reactants={species["B"]: 1, species["*"]: 1},
            products={species["B*"]: 1},
            m2=1e-22,
            amu=1.4e1,
            K=2.87,
            sigma=2,
            sticking=0.015,
            jmol=200 - 1.2 * EO + 1.2 * EN - 0.5 * U,
            do_drc=True,
        ),
        mkmcxx.ARReaction(
            reactants={species["A*"]: 1},
            products={species["B*"]: 1},
            vf=1e13,
            vb=1e13,
            Eaf=100 - 6.0 * EO + 7.5 * EN + 7 * U,
            Eab=600 + 6.0 * EO - 2.5 * EN - 40 * U,
            do_drc=True,
        ),
    ]
    
    return reactions

In [ ]:
# Define grid of points over which we'll do the simulation in EO and EN space.
# Assume that U = 0.05 V vs. SHE.
binding_energy_range = np.arange(-100, 5, 5)  # kJ/mol
EO_range = binding_energy_range
EN_range = binding_energy_range  # Can be different than EO
U = 0.05  # V vs. SHE

grid_points = np.array([[EO, EN] for EO in EO_range for EN in EN_range])

In [ ]:
# Show all the points being considered
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.plot(grid_points[:, 0], grid_points[:, 1], linestyle="", marker=".")

ax.set_xlabel("O binding energy [kJ/mol]")
ax.set_ylabel("N binding energy [kJ/mol]")
ax.set_title("Proposed grid points at which to calculate rates")

In [ ]:
# Filter out all the points that would lead to any negative Arrehenius barriers
def all_AR_barriers_positive(EO, EN, U):
    """
    Test if all barriers in the reaction set are positive
    """

    reactions = generate_reactions(EO, EN, U)

    # Stop as soon as we encounter barriers that are negative
    for reaction in reactions:
        if isinstance(reaction, mkmcxx.ARReaction):
            if not (all([reaction.Eaf > 0, reaction.Eab > 0])):
                return False

    # If we got to this point, then all barriers are positive
    return True

In [ ]:
# Filter the list of grid points
actual_grid_points = np.array(list(
    filter(
        lambda grid_point: all_AR_barriers_positive(
            EO=grid_point[0], EN=grid_point[1], U=U
        ),
        grid_points,
    )
))

In [ ]:
# Now show which grid points will actually be valid
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.plot(actual_grid_points[:, 0], actual_grid_points[:, 1], linestyle="", marker=".")

ax.set_xlabel("O binding energy [kJ/mol]")
ax.set_ylabel("N binding energy [kJ/mol]")
ax.set_title("Actual grid points to be used")

In [ ]:
# Generate dict of reaction sets with EO, EN, and U stored
reaction_sets = [
    {
        "EO": EO,
        "EN": EN,
        "U": U,
        "reactions": generate_reactions(EO=EO, EN=EN, U=U)
    }
    for EO, EN in actual_grid_points
]

### Generate microkinetic simulations

- Now we need to generate a bunch of different simulations, one for each reaction set.

In [ ]:
# Define settings and sequence runs
# See settings docs at https://wiki.mkmcxx.nl/index.php/Keywords_and_settings
settings = {
    "type": "sequencerun",    
    "abstol": 1e-12,
    "reltol": 1e-12,
    "reagents": ["A", "*", "A*"],
    "keycomponents": ["A", "*", "A*"],
    # "heatmap": 1,
    # "npar": 2,
    # "drc": 1,
    # "makeplots": 0,
}

# For now, let's just compare everything at a single temperature
runs = [{"temp": 300, "time": 1e12}]

In [ ]:
# Generate one simulation object per reaction set, keyed with the thermodynamic descriptors
simulations = [
    {
        "EO": reaction_set["EO"],
        "EN": reaction_set["EN"],
        "U": reaction_set["U"],
        "simulation": mkmcxx.MicrokineticSimulation(
            reactions=reaction_set["reactions"],
            settings=settings,
            runs=runs,
            directory=f"""EO-{reaction_set["EO"]}_EN-{reaction_set["EN"]}_U-{reaction_set["U"]}""",
        ),
    }
    for reaction_set in reaction_sets
]

In [ ]:
# Now run all them in parallel

# Worker function (note: must put this in separate .py file and import if on Windows!!)
def worker(simulation_item):

    # Run the simulation object
    simulation_item["simulation"].run(
        args="/home/samueldy/software-images/mkmcxx_2.15.3.sif mkmcxx -i input.mkm".split()
    )

    # Add results into dict and return
    simulation_item["results"] = simulation_item["simulation"].read_results()

    return simulation_item


# Adjust number of processes as necessary
with multiprocessing.Pool(processes=6) as p:
    results = list(tqdm.tqdm(p.imap(worker, simulations), total=len(simulations)))

In [ ]:
# Cache simulations and results
save_to_file(results, "simulation_results.pckl")

### Read grid of results and make TOF plot

- We want to make a TOF plot that is the consumption rate of A in solution.
- So we'll extract this value from each simulation and make a contour plot.

In [ ]:
# Load in results
results = load_from_file("simulation_results.pckl")

In [ ]:
# Need to extract just the conversion rate of A* to B*
# Find this value in the results for each simulation, then aggregate across all simulations
def read_tof_result(result_item):
    """
    Given a single result item, extract just a single rate.
    """
    
    rate = result_item["results"]["range_results"]["rateslog"]["A* -> B*"][0]
    
    simplified_result = {
        "EO": result_item["EO"],
        "EN": result_item["EN"],
        "rate": rate
    }

    return simplified_result

In [ ]:
simplified_results = pd.DataFrame(list(map(read_tof_result, results)))

In [ ]:
# Now make a contour plot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

contourset = ax.tricontourf(
    simplified_results["EO"],
    simplified_results["EN"],
    simplified_results["rate"],
    levels=40,
    cmap="viridis",
)
ax.set_title(fr"""Volcano plot: A* -> B* TOF""")
ax.set_xlabel("O binding energy [kJ/mol]")
ax.set_ylabel("N binding energy [kJ/mol]")

# Add colorbar
fig.colorbar(contourset, label="log10(rate)")